In [1]:
from neo4j import GraphDatabase
from neo4j_graphrag.indexes import create_fulltext_index, create_vector_index,drop_index_if_exists

from neo4j_graphrag.embeddings.openai import OpenAIEmbeddings
from neo4j_graphrag.retrievers import VectorRetriever, HybridCypherRetriever
from neo4j_graphrag.llm import OpenAILLM
from neo4j_graphrag.generation import GraphRAG, RagTemplate

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "auror2025"))

api_key = "ollama"

embedder = OpenAIEmbeddings(
    base_url="http://localhost:11434/v1",
    api_key=api_key,
    model="snowflake-arctic-embed:latest",
)

llm = OpenAILLM(
    base_url="http://localhost:11434/v1",
    model_name="llama3",
    api_key=api_key,
    model_params={"temperature": 0}
)

In [2]:
FULLTEXT_INDEX_NAME = "fulltext_index"

create_fulltext_index(
    driver, FULLTEXT_INDEX_NAME, label="Pokemon", node_properties=["description"]
)

In [3]:
INDEX_NAME = "vector_index"
DIMENSION = 1024

drop_index_if_exists(driver,INDEX_NAME,)

create_vector_index(
    driver,
    INDEX_NAME,
    label="Pokemon",
    embedding_property="vector_property",
    dimensions=DIMENSION,
    similarity_fn="cosine",
)

In [4]:
retriever = VectorRetriever(
    driver,
    index_name="vector_index",
    embedder=embedder,
    return_properties=["name", "description"],
)

query_text = "What is the type and Species of Pikachu"
retriever_result = retriever.search(query_text=query_text, top_k=3)

for item in retriever_result.items:
    print(f"Content: {item.content}")
    print(f"Metadata: {item.metadata}")

rag = GraphRAG(retriever=retriever, llm=llm)

response = rag.search(query_text=query_text, retriever_config={"top_k": 5})

print(response.answer)

Content: {'description': '{"Pokemon":"Pikachu","Type":"Electric","Species":"Mouse Pok\\u00e9mon","Height":0.4,"Weight":6.0,"Abilities":"1. Static, Lightning Rod (hidden ability)","EV Yield":"2 Speed","Catch Rate":"190 (24.8% with Pok\\u00e9Ball, full HP)","Base Friendship":"50 (normal)","Base Exp":"112","Growth Rate":"Medium Fast","Egg Groups":"Fairy, Field","Gender":"50% male, 50% female","Egg Cycles":"10 (2,314\\u20132,570 steps)","HP Base":35,"HP Min":180,"HP Max":274,"Attack Base":55,"Attack Min":103,"Attack Max":229,"Defense Base":40,"Defense Min":76,"Defense Max":196,"Special Attack Base":50,"Special Attack Min":94,"Special Attack Max":218,"Special Defense Base":50,"Special Defense Min":94,"Special Defense Max":218,"Speed Base":90,"Speed Min":166,"Speed Max":306}', 'name': 'Pikachu'}
Metadata: {'score': 0.8129379749298096, 'nodeLabels': ['Pokemon'], 'id': '4:033ff842-49a3-48fd-9efc-e66bd586e692:782'}
Content: {'description': '{"Pokemon":"Partner Pikachu","Type":"Electric","Specie

In [5]:
retrieval_query = """
    MATCH (s:Species)<-[:BELONGS_TO]-(node)
    RETURN
        node.name AS pokemon_name,
        node.description AS pokemon_description, 
    collect(s.name) AS types;
"""

# retrieval_query2 = """ 
# MATCH (s:Species)<-[:BELONGS_TO]-(node)-[:HAS_TYPE]->(t:Type) 
# return node.name as pokemon_name, node.description, collect(s.name) as species, collect(t.name) as types
# """


template = RagTemplate(
    template="""You are an expert in pokemons. Your task is to
    answer the user's question based on the provided context. Use only the
    information within that context.

    Context:
    {context}

    Examples:
    {examples}

    Question:
    {query_text}

    Answer:
    """
)

retriever = HybridCypherRetriever(
    driver,
    vector_index_name="vector_index",
    fulltext_index_name="fulltext_index",
    retrieval_query=retrieval_query,
    embedder=embedder
)

query_text = """which pokemons has height is 0.4 m and weight 6.0 kg"""

retriever_result = retriever.search(query_text=query_text, top_k=5)
for item in retriever_result.items:
    print(f"Content: {item.content}")
    print(f"Metadata: {item.metadata}")

# Initialize the RAG pipeline
rag = GraphRAG(retriever=retriever, llm=llm, prompt_template=template)
response = rag.search(query_text=query_text, retriever_config={"top_k": 5}, return_context=True)
print(response.answer)

Content: <Record pokemon_name='Pikachu' pokemon_description='{"Pokemon":"Pikachu","Type":"Electric","Species":"Mouse Pok\\u00e9mon","Height":0.4,"Weight":6.0,"Abilities":"1. Static, Lightning Rod (hidden ability)","EV Yield":"2 Speed","Catch Rate":"190 (24.8% with Pok\\u00e9Ball, full HP)","Base Friendship":"50 (normal)","Base Exp":"112","Growth Rate":"Medium Fast","Egg Groups":"Fairy, Field","Gender":"50% male, 50% female","Egg Cycles":"10 (2,314\\u20132,570 steps)","HP Base":35,"HP Min":180,"HP Max":274,"Attack Base":55,"Attack Min":103,"Attack Max":229,"Defense Base":40,"Defense Min":76,"Defense Max":196,"Special Attack Base":50,"Special Attack Min":94,"Special Attack Max":218,"Special Defense Base":50,"Special Defense Min":94,"Special Defense Max":218,"Speed Base":90,"Speed Min":166,"Speed Max":306}' types=['Mouse']>
Metadata: None
Content: <Record pokemon_name='Partner Pikachu' pokemon_description='{"Pokemon":"Partner Pikachu","Type":"Electric","Species":"Mouse Pok\\u00e9mon","Hei

In [6]:
query_text = "List all pokemons that have the same type and species of Pikachu"
retriever_result = retriever.search(query_text=query_text, top_k=3)
print(retriever_result)

# Initialize the RAG pipeline
rag = GraphRAG(retriever=retriever, llm=llm)
response = rag.search(query_text=query_text, retriever_config={"top_k": 3})
print(response.answer)

items=[RetrieverResultItem(content='<Record pokemon_name=\'Pikachu\' pokemon_description=\'{"Pokemon":"Pikachu","Type":"Electric","Species":"Mouse Pok\\\\u00e9mon","Height":0.4,"Weight":6.0,"Abilities":"1. Static, Lightning Rod (hidden ability)","EV Yield":"2 Speed","Catch Rate":"190 (24.8% with Pok\\\\u00e9Ball, full HP)","Base Friendship":"50 (normal)","Base Exp":"112","Growth Rate":"Medium Fast","Egg Groups":"Fairy, Field","Gender":"50% male, 50% female","Egg Cycles":"10 (2,314\\\\u20132,570 steps)","HP Base":35,"HP Min":180,"HP Max":274,"Attack Base":55,"Attack Min":103,"Attack Max":229,"Defense Base":40,"Defense Min":76,"Defense Max":196,"Special Attack Base":50,"Special Attack Min":94,"Special Attack Max":218,"Special Defense Base":50,"Special Defense Min":94,"Special Defense Max":218,"Speed Base":90,"Speed Min":166,"Speed Max":306}\' types=[\'Mouse\']>', metadata=None), RetrieverResultItem(content='<Record pokemon_name=\'Castform Snowy Form\' pokemon_description=\'{"Pokemon":"Ca